In [19]:
import os
import numpy as np
import pickle as pkl
import pandas as pd
import stylised_facts_data_utilities.createLOB as createLOB
from collections import defaultdict
folder= '/media/ak/WorkDrive/Data/ProcessedDataForExperiments/'
activityClocksFolder = os.path.join(folder,'ActivityClocks')
folderList = os.listdir(folder)
figures ='/home/ak/Documents/Research/Papers/figures/'

In [20]:
symbol ='SI1'

In [3]:
files =[f for f in folderList if (str(symbol)) in f]
files

['AllQuotes_SI1_Comdty.pkl',
 'SI1_Comdty_Trades',
 'AllTrades_SI1_Comdty.pkl',
 'SI1_Comdty_Quotes']

In [4]:

bidAskfileLoc = "".join((folder, files[0]))
tradefileLoc = "".join((folder, files[2]))
tradefileDict = pkl.load(open(tradefileLoc, "rb"))
bidAskdicts = pkl.load(open(bidAskfileLoc, "rb"))

In [5]:
keys = list(bidAskdicts.keys())

    

In [38]:
activityClocksDict = dict()
for keyIdex in keys:
    print(keyIdex)
    dfTrades = tradefileDict[keyIdex]
    dfQuotes = bidAskdicts[keyIdex]
    LOB = createLOB.createLOBtwoInputs(dfQuotes, dfTrades)
    activityClocksDict[keyIdex]= createLOB.Clocks(createLOB.calcLOB(LOB), 'MicroPrice', 'TradeVolume','DollarVolume',100,25,25)



20180621


/home/ak/virtualenvs/resr3python/lib/python3.6/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


20181011
20180614
20180601
20180627
20180531
20180724
20180518
20180612
20180628
20180516
20180723
20180510
20180928
20181017
20180903
20180517
20181004
20180910
20180926
20180706
20181018
20180424
20180912
20180704
20180625
20181026
20180830
20180514
20180712
20180727
20181008
20181001
20180921
20180829
20181029
20180423
20180806
20180807
20180703
20180711
20180615
20180611
20180619
20180508
20180502
20180425
20180914
20180605
20180821
20180503
20180620
20181003
20180501
20180509
20181023
20180719
20181019
20180618
20180528
20181022
20181024
20180810
20180604
20181010
20180626
20180802
20180709
20180911
20180918
20180420
20180613
20180606
20180507
20180608
20180823
20180920
20180809
20180521
20181025
20180522
20181101
20180824
20180731
20180801
20180813
20180913
20180511
20181009
20181016
20180430
20181005
20180523
20180831
20181031
20180814
20180515
20180827
20180717
20180705
20180828
20180815
20180925
20180919
20180629
20180927
20181002
20180726
20180607
20180702
20181102
20180718
2

In [39]:


pkl.dump(activityClocksDict, open(save_loc, 'wb'), protocol=pkl.HIGHEST_PROTOCOL)

### load pickle here

In [21]:
FileName = "".join(('ActivityClocks_', symbol, '.pkl'))

load_loc = os.path.join(activityClocksFolder, FileName)


In [23]:
infile = open(load_loc ,'rb')
activityClockFile = pkl.load(infile)
infile.close()

In [25]:
keys = list(activityClockFile.keys())

In [26]:

activityClockFile[keys[1]].volume_bar_df().columns.values

100%|██████████| 655729/655729 [00:00<00:00, 2291429.13it/s]


array(['BidSize', 'QuoteTime', 'type_x', 'BestBid', 'TradeTime',
       'AskSize', 'time_y', 'type_y', 'BestAsk', 'TradeVolume', 'TradeId',
       'TradePrice', 'TimeStamp', 'milliSeconds', 'DollarVolume',
       'MicroPrice', 'Duration', 'PriceChange', 'LogReturn', 'BidOffer',
       'L1PriceImbalance', 'L1SizeRatio', 'MicroPricePctChange',
       'FwdMPChange_1', 'FwdMPChange_5', 'LogTradeSize'], dtype=object)

In [ ]:
from statsmodels.tsa.seasonal import STL
%matplotlib inline
import fathon
from fathon import fathonUtils as fu

#### get all the information clocks sorted

In [ ]:
volumeBarDF = activityClocksDict[keys[1]].volume_bar_df()
tickBarDF = activityClocksDict[keys[1]].tick_bar_df()
dollarVolumeBarDF = activityClocksDict[keys[1]].dollar_bar_df()
originalDF = activityClocksDict[keys[1]].original_df()

In [ ]:
##pull out variable of choice
variable = 'L1PriceImbalance'
volumeBarVariable = np.asanyarray(volumeBarDF[str(variable)])
tickBarVariable = np.asanyarray(tickBarDF[str(variable)])
dollarVolumeBarVariable = np.asanyarray(tickBarDF[str(variable)])
originalVariable = originalDF[str(variable)]

In [ ]:
len(dollarVolumeBarVariabe)

In [ ]:

a = fu.toAggregated(dollarVolumeBarVariable)


In [ ]:
pymfdfa = fathon.MFDFA(a)


In [ ]:
winSizes = fu.linRangeByStep(10, 2000)
qs = np.arange(-3, 4, 0.1)
revSeg = True
polOrd = 1
n, F = pymfdfa.computeFlucVec(winSizes, qs, revSeg=revSeg, polOrd=polOrd)


In [ ]:
list_H, list_H_intercept = pymfdfa.fitFlucVec()

In [ ]:
plt.plot(np.log(n), np.log(F[0, :]), 'ro')
plt.plot(np.log(n), list_H_intercept[0]+list_H[0]*np.log(n), '--', label='h_{:.1f} = {:.2f}'.format(qs[0], list_H[0]))
half_idx = int(len(qs)/2)
plt.plot(np.log(n), np.log(F[half_idx, :]), 'co')
plt.plot(np.log(n), list_H_intercept[half_idx]+list_H[half_idx]*np.log(n),
         '--', label='h_{:.1f} = {:.2f}'.format(qs[half_idx], list_H[half_idx]))
plt.plot(np.log(n), np.log(F[-1, :]), 'yo')
plt.plot(np.log(n), list_H_intercept[-1]+list_H[-1]*np.log(n), '--',
         label='h_{:.1f} = {:.2f}'.format(qs[-1], list_H[-1]))
plt.xlabel('ln(n)', fontsize=14)
plt.ylabel('ln(F(n))', fontsize=14)
plt.title('MFDFA', fontsize=14)
plt.legend(loc=0, fontsize=14)

title = 'MDFDA_'+str(variable)+str(symbol)+'_plot1.png'
fileLocation = os.path.join(figures, title)
plt.savefig(fileLocation, dpi = 150 )

In [ ]:
plt.plot(qs, list_H, 'ro-')
plt.xlabel('q', fontsize=14)
plt.ylabel('h(q)', fontsize=14)
plt.title('h(q)', fontsize=14)

In [ ]:
tau = pymfdfa.computeMassExponents()

In [27]:
tau = pymfdfa.computeMassExponents()

#### get all the information clocks sorted

In [28]:
volumeBarDF = activityClocksDict[keys[1]].volume_bar_df()
tickBarDF = activityClocksDict[keys[1]].tick_bar_df()
dollarVolumeBarDF = activityClocksDict[keys[1]].dollar_bar_df()
originalDF = activityClocksDict[keys[1]].original_df()

100%|██████████| 655729/655729 [00:00<00:00, 2576102.39it/s]


In [34]:
##pull out variable of choice
variable = 'L1PriceImbalance'
volumeBarVariable = np.asanyarray(volumeBarDF[str(variable)])
tickBarVariable = np.asanyarray(tickBarDF[str(variable)])
dollarVolumeBarVariable = np.asanyarray(tickBarDF[str(variable)])
originalVariable = originalDF[str(variable)]

In [33]:
len(dollarVolumeBarVariabe)

65528

In [35]:

a = fu.toAggregated(dollarVolumeBarVariable)


In [36]:
pymfdfa = fathon.MFDFA(a)


In [37]:
winSizes = fu.linRangeByStep(10, 2000)
qs = np.arange(-3, 4, 0.1)
revSeg = True
polOrd = 1
n, F = pymfdfa.computeFlucVec(winSizes, qs, revSeg=revSeg, polOrd=polOrd)


In [ ]:
list_H, list_H_intercept = pymfdfa.fitFlucVec()

In [ ]:
plt.plot(np.log(n), np.log(F[0, :]), 'ro')
plt.plot(np.log(n), list_H_intercept[0]+list_H[0]*np.log(n), '--', label='h_{:.1f} = {:.2f}'.format(qs[0], list_H[0]))
half_idx = int(len(qs)/2)
plt.plot(np.log(n), np.log(F[half_idx, :]), 'co')
plt.plot(np.log(n), list_H_intercept[half_idx]+list_H[half_idx]*np.log(n),
         '--', label='h_{:.1f} = {:.2f}'.format(qs[half_idx], list_H[half_idx]))
plt.plot(np.log(n), np.log(F[-1, :]), 'yo')
plt.plot(np.log(n), list_H_intercept[-1]+list_H[-1]*np.log(n), '--',
         label='h_{:.1f} = {:.2f}'.format(qs[-1], list_H[-1]))
plt.xlabel('ln(n)', fontsize=14)
plt.ylabel('ln(F(n))', fontsize=14)
plt.title('MFDFA', fontsize=14)
plt.legend(loc=0, fontsize=14)

title = 'MDFDA_'+str(variable)+str(symbol)+'_plot1.png'
fileLocation = os.path.join(figures, title)
plt.savefig(fileLocation, dpi = 150 )

In [ ]:
plt.plot(qs, list_H, 'ro-')
plt.xlabel('q', fontsize=14)
plt.ylabel('h(q)', fontsize=14)
plt.title('h(q)', fontsize=14)

In [ ]:
tau = pymfdfa.computeMassExponents()